<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [5]</a>'.</span>

In [1]:
# Parameters
dataset_path = "temp\\eac12fd1-c872-46c5-b1a0-9466ecbd6ca7.csv"
algorithm = "bayesian"
model_output_path = "saved_models/Bayesian_Opt_Catboost_1742120361.cbm"


In [2]:
#PIP INSTALLATION
print('---1---')
%pip install pandas
print('---2---')
%pip install seaborn
print('---3---')
%pip install scikit-learn
print('---4---')
%pip install catboost
print('---5---')
%pip install ipywidgets
print('---6---')
%pip install pandas openpyxl
print('---7---')
%pip install bayesian-optimization
print('---8---')
%pip install fsspec
print('---9---')
%pip install huggingface_hub
print('---10---')
%pip install datasets


---1---


Note: you may need to restart the kernel to use updated packages.
---2---



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
---3---



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


...


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
---10---



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from catboost import CatBoostClassifier, Pool

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_curve, roc_auc_score, precision_recall_curve, average_precision_score,
    confusion_matrix
)
from sklearn.model_selection import StratifiedKFold
from bayes_opt import BayesianOptimization
from typing import List, Optional
from sklearn.model_selection import train_test_split


from datasets import load_dataset


import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [4]:
dataset_path = None
algorithm = 'catboost'

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [5]:
df = pd.read_csv(dataset_path)
X = df.drop(columns=['readmitted'])
y = df['readmitted'].values

ValueError: Invalid file path or buffer object type: <class 'NoneType'>

In [ ]:
cat_features = X.select_dtypes(include=['object']).columns.tolist()
X[cat_features] = X[cat_features].astype(str)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Final CatboostBayesianOptimzer

In [ ]:
gpu_available = "NVIDIA_VISIBLE_DEVICES" in os.environ or "CUDA_VISIBLE_DEVICES" in os.environ
task_type = 'GPU' if gpu_available else 'CPU'

class CatBoostBayesianOptimizer:
    def __init__(self, X_train, y_train, cat_features, eval_metric, n_splits, random_state=42, gpu_id=0):
        self.X_train = X_train
        self.y_train = y_train
        self.cat_features = cat_features
        self.eval_metric = eval_metric
        self.n_splits = n_splits
        self.random_state = random_state
        self.gpu_id = gpu_id
        
        self._validate_inputs()
        
    def _validate_inputs(self):
        if not isinstance(self.X_train, pd.DataFrame):
            raise TypeError("X_train must be a pandas DataFrame")
        
        if not all(col in self.X_train.columns for col in self.cat_features):
            raise ValueError("Some categorical features not found in X_train")
            
        if self.eval_metric not in ['f1', 'auc', 'accuracy']:
            raise ValueError("eval_metric must be one of: 'f1', 'auc', 'accuracy'")
    
    #This is also hyperparameters for catboost but it cant be applied with bayesian optimization
    def _get_base_params(self, custom_params=None):
        """Get base CatBoost parameters"""
        base_params = {
            'random_state': self.random_state,
            'verbose': False,
            'task_type': task_type,
            'devices': f'{self.gpu_id}' if task_type == 'GPU' else None,
            'loss_function': 'Logloss',
            'eval_metric': 'Logloss'
        }
        return {**base_params, **custom_params} if custom_params else base_params
    
    def _get_metric_score(self, y_true, y_pred, y_prob=None):
        if self.eval_metric == 'f1':
            return f1_score(y_true, y_pred, average='weighted')
        elif self.eval_metric == 'auc':
            if len(np.unique(y_true)) > 2:
                return roc_auc_score(y_true, y_prob, multi_class='ovr')
            return roc_auc_score(y_true, y_prob[:, 1])
        return accuracy_score(y_true, y_pred)
    
    def _evaluate_model(self, model, X_val, y_val):
        """Evaluate model performance"""
        if self.eval_metric == 'auc':
            y_prob = model.predict_proba(X_val)
            return self._get_metric_score(y_val, np.argmax(y_prob, axis=1), y_prob)
        y_pred = model.predict(X_val)
        return self._get_metric_score(y_val, y_pred)
        
    #This is the 1st step to add another Parameters... you can add in here
    def _objective(self, iterations, learning_rate, depth, l2_leaf_reg, border_count):
        try:
            params = self._get_base_params({
                'iterations': int(iterations),
                'learning_rate': float(learning_rate),
                'depth': int(depth),
                'l2_leaf_reg': float(l2_leaf_reg),
                'border_count': int(border_count),
            })
            
            X_train, X_val, y_train, y_val = train_test_split(
                self.X_train, self.y_train,
                test_size=0.2,
                random_state=self.random_state,
                stratify=self.y_train
            )
            
            train_pool = Pool(X_train, label=y_train, cat_features=self.cat_features)
            val_pool = Pool(X_val, label=y_val, cat_features=self.cat_features)
            
            model = CatBoostClassifier(**params)
            model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=50, verbose=False)
            
            return self._evaluate_model(model, X_val, y_val)
                
        except Exception as e:
            print(f"Error in optimization: {str(e)}")
            return 0.0
        
    def optimize(self, n_iter=50, n_init_points=10):
        if n_iter <= n_init_points:
            raise ValueError("n_iter must be greater than n_init_points")
        
        #After adding the parameters you set the value for the parameters
        pbounds = {
            'iterations': (800, 1000),
            'learning_rate': (0.025, 0.035),
            'depth': (7, 9),
            'l2_leaf_reg': (3.0, 5.0),
            'border_count': (60, 85),
        }
        
        optimizer = BayesianOptimization(
            f=self._objective,
            pbounds=pbounds,
            random_state=self.random_state
        )
        
        optimizer.maximize(init_points=n_init_points, n_iter=n_iter)
        
        best_params = optimizer.max['params']
        print("\nBest parameters found:")
        for param, value in best_params.items():
            if param in ['depth', 'border_count', 'iterations']:
                print(f"{param}: {int(value)}")
            else:
                print(f"{param}: {value:.4f}")
        print(f"\nBest CV {self.eval_metric} score: {optimizer.max['target']:.4f}")
    
        return best_params
    
    def Final_model(self, best_params):
        """Use SKCV to evaluate and choose the best iteration count, then train a final model"""
        # Prepare cross-validation folds
        skf = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=self.random_state)
        fold_scores = []
        best_iterations = []
        
        # Create base parameters from best Bayesian optimization results
        model_params = self._get_base_params({
            'iterations': int(best_params['iterations']),
            'learning_rate': best_params['learning_rate'],
            'depth': int(best_params['depth']),
            'l2_leaf_reg': best_params['l2_leaf_reg'],
            'border_count': int(best_params['border_count'])
        })
        
        print(f"\nPerforming {self.n_splits}-fold CV to determine optimal iterations...")
        
        for fold, (train_idx, val_idx) in enumerate(skf.split(self.X_train, self.y_train), 1):
            X_fold_train = self.X_train.iloc[train_idx]
            y_fold_train = self.y_train[train_idx]
            X_fold_val = self.X_train.iloc[val_idx]
            y_fold_val = self.y_train[val_idx]
            
            train_pool = Pool(X_fold_train, label=y_fold_train, cat_features=self.cat_features)
            val_pool = Pool(X_fold_val, label=y_fold_val, cat_features=self.cat_features)
            
            # Train model with early stopping
            model = CatBoostClassifier(**model_params)

            #You can change the final model here
            model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=50, verbose=100)
            
            # Store the best iteration number
            best_iterations.append(model.get_best_iteration())
            
            # Evaluate on validation set
            score = self._evaluate_model(model, X_fold_val, y_fold_val)
            print(f"Fold {fold} {self.eval_metric} score: {score:.4f}, Best iteration: {model.get_best_iteration()}")
            
            fold_scores.append(score)
        
        mean_score = np.mean(fold_scores)
        std_score = np.std(fold_scores)
        
        # Calculate the median best iteration across folds
        optimal_iterations = int(np.median(best_iterations)) if np.std(best_iterations) > 5 else int(np.mean(best_iterations))
        
        print(f"\nCV Results:")
        print(f"Average {self.eval_metric} score: {mean_score:.4f} ± {std_score:.4f}")
        print(f"Optimal iterations (median across folds): {optimal_iterations}")
        
        # Train final model on split dataset with validation set
        X_train_final, X_val_final, y_train_final, y_val_final = train_test_split(
            self.X_train, self.y_train,
            test_size=0.1,  # 20% validation set
            random_state=self.random_state,
            stratify=self.y_train
        )
        
        train_pool = Pool(X_train_final, label=y_train_final, cat_features=self.cat_features)
        val_pool = Pool(X_val_final, label=y_val_final, cat_features=self.cat_features)
        
        final_model_params = model_params.copy()
        final_model_params['iterations'] = optimal_iterations
        
        print(f"\nTraining final model on training dataset with {optimal_iterations} iterations...")
        
        final_model = CatBoostClassifier(**final_model_params)
        final_model.fit(train_pool, eval_set=val_pool, verbose=100)
        
        if False:  # Set to True if you want the full retrain without validation
            print("Retraining on full dataset without validation...")
            full_model_params = final_model_params.copy()
            full_train_pool = Pool(self.X_train, label=self.y_train, cat_features=self.cat_features)
            full_model = CatBoostClassifier(**full_model_params)
            full_model.fit(full_train_pool, verbose=100)
            return full_model, mean_score, std_score
        
        return final_model, mean_score, std_score

    def save_model(self, model, filepath):
        """Save the trained CatBoost model to disk"""
        try:
            model.save_model(filepath)
            print(f"Model successfully saved to {filepath}")
            return True
        except Exception as e:
            print(f"Error saving model: {str(e)}")
            return False

    

In [ ]:
class FinalModelEvaluator:
    def __init__(
        self,
        model,
        X_train: pd.DataFrame,
        y_train: np.ndarray,
        X_test: pd.DataFrame,
        y_test: np.ndarray,
        cat_features: list
    ):
        self.model = model
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.cat_features = cat_features
        
        # Calculate predictions for the final model
        self.y_pred = model.predict(X_test)
        self.y_pred_proba = model.predict_proba(X_test)

    def plot_learning_curve(self, figsize: tuple = (12, 5)):
        """Plot learning curve for the final model."""
        plt.figure(figsize=figsize)
        
        evals_result = self.model.get_evals_result()
        train_loss = evals_result['learn']['Logloss']
        
        plt.plot(train_loss, label='Training Loss')
        
        # Check for validation data under either 'validation' or 'test' keys
        if 'validation' in evals_result:
            val_loss = evals_result['validation']['Logloss']
            plt.plot(val_loss, label='Validation Loss')
        elif 'test' in evals_result:
            val_loss = evals_result['test']['Logloss']
            plt.plot(val_loss, label='Validation Loss')
            
        plt.title('Learning Curve (Final Model)')
        plt.xlabel('Iterations')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        plt.show()

    def plot_feature_importance(self, top_n: int = 20, figsize: tuple = (10, 6)):
        """Plot feature importance of the final model."""
        importance = pd.DataFrame({
            'Feature': self.X_train.columns,
            'Importance': self.model.get_feature_importance()
        }).sort_values('Importance', ascending=False)
        
        if top_n:
            importance = importance.head(top_n)
            
        plt.figure(figsize=figsize)
        plt.barh(importance['Feature'], importance['Importance'])
        plt.title(f'Top {top_n} Feature Importance (Final Model)' if top_n else 'Feature Importance')
        plt.xlabel('Importance Score')
        plt.tight_layout()
        plt.show()

    def plot_roc_curve(self, figsize: tuple = (8, 6)):
        """Plot ROC curve for the final model."""
        plt.figure(figsize=figsize)
        
        fpr, tpr, _ = roc_curve(self.y_test, self.y_pred_proba[:, 1])
        auc = roc_auc_score(self.y_test, self.y_pred_proba[:, 1])
        
        plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {auc:.3f})')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curve (Final Model)')
        plt.legend(loc="lower right")
        plt.grid(True)
        plt.show()

    def plot_confusion_matrix(self, figsize: tuple = (8, 6)):
        """Plot confusion matrix for the final model."""
        plt.figure(figsize=figsize)
        
        cm = confusion_matrix(self.y_test, self.y_pred)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix (Final Model)')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.show()

    def get_metrics(self) -> dict:
        """Calculate and return metrics for the final model."""
        metrics = {
            'Accuracy': accuracy_score(self.y_test, self.y_pred),
            'Precision': precision_score(self.y_test, self.y_pred),
            'Recall': recall_score(self.y_test, self.y_pred),
            'F1 Score': f1_score(self.y_test, self.y_pred),
            'ROC AUC': roc_auc_score(self.y_test, self.y_pred_proba[:, 1])
        }
        
        return metrics

    def print_metrics(self):
        """Print all metrics in a formatted way."""
        metrics = self.get_metrics()
        
        print("\nFinal Model Performance Metrics:")
        print("-------------------------------")
        for metric, value in metrics.items():
            print(f"{metric}: {value:.4f}")

    def compare_with_cv_metrics(self, cv_mean_metrics, cv_std_metrics):
        """Compare final model metrics with cross-validation metrics."""
        final_metrics = self.get_metrics()
        
        print("\nMetrics Comparison (Final Model vs. Cross-Validation):")
        print("-------------------------------------------------")
        for metric in final_metrics.keys():
            final_value = final_metrics[metric]
            if metric in cv_mean_metrics:
                cv_mean = cv_mean_metrics[metric]
                cv_std = cv_std_metrics[metric]
                diff = final_value - cv_mean
                print(f"{metric}:")
                print(f"  - Final Model: {final_value:.4f}")
                print(f"  - CV Mean:     {cv_mean:.4f} ± {cv_std:.4f}")
                print(f"  - Difference:  {diff:.4f} ({diff/cv_mean*100:.2f}%)")
            else:
                print(f"{metric}: {final_value:.4f} (CV metric not available)")

    def evaluate_all(self):
        """Run all evaluations and plots."""
        self.print_metrics()
        self.plot_learning_curve()
        self.plot_feature_importance()
        self.plot_roc_curve()
        self.plot_confusion_matrix()

In [ ]:
optimizer = CatBoostBayesianOptimizer(
    X_train=X_train,
    y_train=y_train,
    cat_features=cat_features,
    eval_metric="auc", 
    n_splits=5,
    random_state=42,
    gpu_id=0
)

#We can also change the iterations of the bayesion and the best points
best_params = optimizer.optimize(n_iter=10, n_init_points=2)



In [ ]:
# Select best model using SKCV and train final model
final_model, cv_score, cv_std = optimizer.Final_model(best_params)

In [ ]:
# Now evaluate the final model
evaluator = FinalModelEvaluator(
    model=final_model,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    cat_features=cat_features
)

# Run all evaluations
evaluator.evaluate_all()


In [ ]:
os.makedirs('saved_models', exist_ok=True)
final_model.save_model('saved_models/Bayesian_Opt_Catboost.cbm')